# CSCI_544 - HW2 - Sukavanan Nanjundan

Note to the grader: If you want to run the whole code, I suggesst skipping the model training process and use the pretrained model files that I've included. If you run into memory errors anywhere, please just restart the kernel and run from the previous checkpoints (which I have mentioned). You could also manually increase the virtual memory of the system just for the duration of running this file to ensure a smoother process. I might delete some variables at some point to reduce memory usage and speed up the training process, please disregard those.

In [1]:
import pandas as pd
import numpy as np
import random
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import copy
import warnings
import gc
import pickle as pkl
from collections import Counter
from sklearn.utils import class_weight
warnings.filterwarnings('ignore')

First, we are reading the TSV file and storing it as a dataframe.

In [2]:
data = pd.read_csv("data/amazon_reviews_us_Kitchen_v1_00.tsv", sep = '\t', error_bad_lines = False, warn_bad_lines = False)

Now, dropping the rows with NAN values.

In [3]:
data = data.dropna()

Dropping all the columns except the review_body and the star_rating columns.

In [4]:
columns = list(data.columns)
columns.remove('review_body')
columns.remove('star_rating')
df = data.drop(columns, axis = 1)

Cleaning the data by converting it to lowercase, removing urls and html data and also by removing extra white spaces between words.

In [5]:
df.review_body = df.review_body.str.lower()
df.review_body = df.review_body.replace("http\S+", "", regex = True).replace("www\S+", "", regex = True).replace("<.*?>", "", regex = True)
df.review_body = df.review_body.replace("[^a-zA-Z ]", " ", regex = True)
df.review_body = df.review_body.replace(" +", " ", regex = True)

Once again, dropping any null or NAN values that might have arised after the data cleaning process and storing the data for easier access in the future.

In [6]:
df = df.dropna()
df.isnull().any()
df.to_csv("data/amazon_reviews_cleaned.csv", index = False)

In [ ]:
df = pd.read_csv("data/amazon_reviews_cleaned.csv")
df = df.dropna()

Now, we select 50,0000 samples for each rating and then use that as our new dataset.

In [7]:
rat_1 = df[df.star_rating == 1].sample(n = 50000, random_state = 42)
rat_2 = df[df.star_rating == 2].sample(n = 50000, random_state = 42)
rat_3 = df[df.star_rating == 3].sample(n = 50000, random_state = 42)
rat_4 = df[df.star_rating == 4].sample(n = 50000, random_state = 42)
rat_5 = df[df.star_rating == 5].sample(n = 50000, random_state = 42)

Concatenating the samples to form one dataframe and replacing the star ratings with the values 0, 1, 2 representing negative (0, 1 ratings), positive (4, 5 ratings) and neutral (3 rating) sentiments of the reviews.

In [73]:
frames = [rat_1, rat_2, rat_3, rat_4, rat_5]
df_new = pd.concat(frames, ignore_index = True)
df_new = df_new.replace({'star_rating': {1: 0, 2: 0, 4: 1, 5: 1, 3: 2}})

In [11]:
df_new = df_new.dropna()

# Creating the word2vec model

##### Checkpoint - 1

In [2]:
import gensim.downloader as api
w2v_google = api.load('word2vec-google-news-300')

As you can see, the google word2vec model has been trained on a large and meaningful data such that the vector represantations of the words are semantically correct, as corroborated by the examples below.

In [18]:
w2v_google.most_similar(positive = ['king', 'woman'], negative = ['man'], topn = 1)

[('queen', 0.7118193507194519)]

In [19]:
w2v_google.similarity("excellent", "outstanding")

0.55674857

In [3]:
from gensim import utils
import gensim.models

Now, we define a corpus class that iterates over each review in our dataframe and gives that as input to the word2vec model for training.

In [76]:
class MyCorpus:
    def __iter__(self):
        for i in range(len(df_new)):
            yield utils.simple_preprocess(str(df_new.iloc[[i]].review_body.values[0]))

sentences = MyCorpus()
w2v_model = gensim.models.Word2Vec(sentences = sentences, vector_size = 300, min_count = 10, window = 11)

In [77]:
w2v_model.save("model/gensim_w2v_amazon_reviews")

In [4]:
w2v_model = gensim.models.Word2Vec.load("model/gensim_w2v_amazon_reviews")

As you can see, the word2vec model trained just using the 250K reviews has not been able to produce the same level of semantic correctness when compared to the google word2vec model as proven by the 'king' + 'woman' - 'man' example. But, in the subsequent examples of comparing the similarity between "outstanding" and "excellent", and 'utensil' + 'sharp', the model seems to perform well. This might be due to the fact that the context of the latter examples would be closely related to that of  a review and the model would have been able to capture their semantic similarities with more accuracy.

In [78]:
w2v_model.wv.most_similar(positive = ['king', 'woman'], negative = ['man'], topn = 10)

[('shams', 0.44163042306900024),
 ('qt', 0.41580283641815186),
 ('cambro', 0.41324180364608765),
 ('quart', 0.4047726094722748),
 ('americolor', 0.3984954357147217),
 ('isi', 0.39528515934944153),
 ('arthur', 0.39269858598709106),
 ('elite', 0.3870254158973694),
 ('cd', 0.38650548458099365),
 ('jumbo', 0.3864506185054779)]

In [79]:
w2v_model.wv.most_similar(positive = ["utensil", "sharp"], topn = 5)

[('scissors', 0.5846854448318481),
 ('knife', 0.581675112247467),
 ('knifes', 0.5791575312614441),
 ('utensils', 0.5731496214866638),
 ('serrated', 0.5610765814781189)]

In [80]:
w2v_model.wv.similarity("excellent", "outstanding")

0.77758265

Performing data pre-processing tasks such as contractions, stop-word removal and lemmatization. Pre-processing helps in improving the accuracy.

In [12]:
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Performing Contractions on the reviews
df_new.review_body = df_new.review_body.replace(df_new.review_body.values[0], contractions.fix(df_new.review_body.values[0]).lower())

#Performing stop word removal on the reviews
stop = stopwords.words("english")
df_new.review_body = df_new.review_body.apply(lambda x: " ".join([word for word in x.split() if word not in stop]))

#Performing Lemmatization on the reviews
lemmatizer = WordNetLemmatizer()
df_new.review_body = df_new.review_body.apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split(" ")]))

Saving the dataframe for easier access in the future.

In [13]:
df_new = df_new.dropna()
df_new.to_csv("data/50k_samples.csv", index = False)

##### Checkpoint - 2

In [5]:
df_new = pd.read_csv("data/50k_samples.csv")
df_new = df_new.dropna()

Below, is the function that returns the average of the vectors of each word that is produced by the given word2vec model (google or amazon reviews). We ignore the words that are not present in the given model using the try, except block.

In [84]:
def compute_average_w2v(sentence, model):
    result_vector = np.ndarray(shape = (300,), buffer = np.zeros((300,)), dtype = float)
    removed = 0
    if model == 'google':
        for word in sentence.split(" "):
            try:
                result_vector = result_vector + w2v_google[word]
            except KeyError:
                removed += 1
                continue
    else:
        for word in sentence.split(" "):
            try:
                result_vector = result_vector + w2v_model.wv[word]
            except KeyError:
                removed += 1
                continue
    return list(result_vector / (len(sentence) - removed))

Now, we create a list of tuples where the first element is the average vector of the given review as calculated by the above mentioned function and the second element is the label for that review. We do this for both the google and the amazon word2vec models.

In [85]:
google = [(compute_average_w2v(df_new.iloc[[x]].review_body.values[0], "google"), df_new.iloc[[x]].star_rating.values[0]) for x in range(len(df_new))]
amazon = [(compute_average_w2v(df_new.iloc[[x]].review_body.values[0], "amazon"), df_new.iloc[[x]].star_rating.values[0]) for x in range(len(df_new))]

Then, we remove any null or NAN values from the list.

In [86]:
google = [google[x] for x in range(len(google)) if np.isnan(google[x][0]).any() == False]
amazon = [amazon[x] for x in range(len(amazon)) if np.isnan(amazon[x][0]).any() == False]

Now, we convert the list of tuples into a dataframe and rename the columns as vector and label.

In [87]:
google_df = pd.DataFrame(google)
amazon_df = pd.DataFrame(amazon)
google_df = google_df.rename(columns = {0: "vector", 1: "label"})
amazon_df = amazon_df.rename(columns = {0: "vector", 1: "label"})

Then, we create 300 columns to store the values of each individual dimension of the word vector.

In [88]:
google_df[['vector' + '_{}'.format(x) for x in range(300)]] = pd.DataFrame(google_df.vector.to_list(), index = google_df.index)
amazon_df[['vector' + '_{}'.format(x) for x in range(300)]] = pd.DataFrame(amazon_df.vector.to_list(), index = amazon_df.index)
google_df = google_df.drop(['vector'], axis = 1)
amazon_df = amazon_df.drop(['vector'], axis = 1)

Storing the dataframes as CSV files for easier access in the future.

In [92]:
google_df.to_csv("data/google_model.csv")
amazon_df.to_csv("data/amazon_model.csv")

##### Checkpoint - 3

In [33]:
google_df = pd.read_csv("data/google_model.csv")
amazon_df = pd.read_csv("data/amazon_model.csv")

In [93]:
google_binary = google_df[google_df["label"] != 2]
amazon_binary = amazon_df[amazon_df["label"] != 2]

In [94]:
x_google = google_binary.drop(['label'], axis = 1).values
y_google = google_binary.label.values
x_amazon = amazon_binary.drop(['label'], axis = 1).values
y_amazon = amazon_binary.label.values

In [96]:
bin_df = df_new[df_new["star_rating"] != 2]
x_tfidf = bin_df.review_body
y_tfidf = bin_df.star_rating

Splitting the data into training and testing sets. 80/20 split.

In [97]:
x_google_train, x_google_test, y_google_train, y_google_test = train_test_split(x_google, y_google, test_size=0.2, random_state=42)
x_amazon_train, x_amazon_test, y_amazon_train, y_amazon_test = train_test_split(x_amazon, y_amazon, test_size=0.2, random_state=42)
x_tfidf_train, x_tfidf_test, y_tfidf_train, y_tfidf_test = train_test_split(x_tfidf, y_tfidf, test_size=0.2, random_state=42)

Initializing a TF_IDF model and training it with our binary class dataset.

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_extractor = TfidfVectorizer()
x_tfidf_train = tf_idf_extractor.fit_transform(x_tfidf_train)
x_tfidf_test = tf_idf_extractor.transform(x_tfidf_test)

In [100]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
perceptron = Perceptron()

## Simple Models

In [101]:
#Perceptron trained using Google W2V features
perceptron.fit(x_google_train, y_google_train)
y_pred_test = perceptron.predict(x_google_test)
print("Statistics of perceptron model trained using Google Word2Vec features: ")
print("Testing Accuracy: ", accuracy_score(y_google_test, y_pred_test))
print("Testing Precision: ", precision_score(y_google_test, y_pred_test))
print("Testing Recall: ", recall_score(y_google_test, y_pred_test))
print("Testing F1 Score: ", f1_score(y_google_test, y_pred_test))

Statistics of perceptron model trained using Google Word2Vec features: 
Testing Accuracy:  0.613075
Testing Precision:  0.9622448979591837
Testing Recall:  0.23566751636927075
Testing F1 Score:  0.3786084233348055


In [102]:
#Perceptron trained using Amazon W2V features
perceptron.fit(x_amazon_train, y_amazon_train)
y_pred_test = perceptron.predict(x_amazon_test)
print("Statistics of perceptron model trained using Amazon Word2Vec features: ")
print("Testing Accuracy: ", accuracy_score(y_amazon_test, y_pred_test))
print("Testing Precision: ", precision_score(y_amazon_test, y_pred_test))
print("Testing Recall: ", recall_score(y_amazon_test, y_pred_test))
print("Testing F1 Score: ", f1_score(y_amazon_test, y_pred_test))

Statistics of perceptron model trained using Amazon Word2Vec features: 
Testing Accuracy:  0.7914145707285364
Testing Precision:  0.7893648449039882
Testing Recall:  0.7977310046275563
Testing F1 Score:  0.7935258742297127


In [103]:
#Perceptron trained using TF_IDF W2V features
perceptron.fit(x_tfidf_train, y_tfidf_train)
y_pred_test = perceptron.predict(x_tfidf_test)
print("Statistics of perceptron model trained using TF_IDF features: ")
print("Testing Accuracy: ", accuracy_score(y_tfidf_test, y_pred_test))
print("Testing Precision: ", precision_score(y_tfidf_test, y_pred_test))
print("Testing Recall: ", recall_score(y_tfidf_test, y_pred_test))
print("Testing F1 Score: ", f1_score(y_tfidf_test, y_pred_test))

Statistics of perceptron model trained using TF_IDF features: 
Testing Accuracy:  0.82335
Testing Precision:  0.8307857471499412
Testing Recall:  0.8122657070025491
Testing F1 Score:  0.8214213505863324


As we can see the the perceptron model performs the best when the TF_IDF features are used. This might be due to the fact that the perceptron is inherently a very simple model and wasn't able to capture the complex dependencies between the word2vec features and the labels in our dataset. We also notice that the amazon reviews model outperfoms the google word2vec model. This is due to the fact that the we trained our word2vec model on the given dataset whereas the google model was trained on a different dataset. Yet, the google model does perform decently well in comparison to our model. We should be able to see an improvement in the performance of both the word2vec features when a model that can handle complex dependencies is used.

In [104]:
from sklearn.svm import LinearSVC
svm = LinearSVC(dual = False)

In [105]:
#SVM model trained using Google Word2Vec features
svm.fit(x_google_train, y_google_train)
y_pred_train = svm.predict(x_google_train)
y_pred_test = svm.predict(x_google_test)
print("Statistics of SVM model trained using Google Word2Vec features: ")
print("Testing Accuracy: ", accuracy_score(y_google_test, y_pred_test))
print("Testing Precision: ", precision_score(y_google_test, y_pred_test))
print("Testing Recall: ", recall_score(y_google_test, y_pred_test))
print("Testing F1 Score: ", f1_score(y_google_test, y_pred_test))

Statistics of SVM model trained using Google Word2Vec features: 
Testing Accuracy:  0.817625
Testing Precision:  0.8347024749868351
Testing Recall:  0.7922727045534063
Testing F1 Score:  0.8129343282816626


In [106]:
#SVM model trained using Amazon Word2Vec features
svm.fit(x_amazon_train, y_amazon_train)
y_pred_train = svm.predict(x_amazon_train)
y_pred_test = svm.predict(x_amazon_test)
print("Statistics of SVM model trained using Google Word2Vec features: ")
print("Testing Accuracy: ", accuracy_score(y_amazon_test, y_pred_test))
print("Testing Precision: ", precision_score(y_amazon_test, y_pred_test))
print("Testing Recall: ", recall_score(y_amazon_test, y_pred_test))
print("Testing F1 Score: ", f1_score(y_amazon_test, y_pred_test))

Statistics of SVM model trained using Google Word2Vec features: 
Testing Accuracy:  0.83989199459973
Testing Precision:  0.8487951500331142
Testing Recall:  0.8290292083395532
Testing F1 Score:  0.8387957508936214


In [107]:
#SVM model trained using TF_IDF features
svm.fit(x_tfidf_train, y_tfidf_train)
y_pred_train = svm.predict(x_tfidf_train)
y_pred_test = svm.predict(x_tfidf_test)
print("Statistics of SVM model trained using TF_IDF features: ")
print("Testing Accuracy: ", accuracy_score(y_tfidf_test, y_pred_test))
print("Testing Precision: ", precision_score(y_tfidf_test, y_pred_test))
print("Testing Recall: ", recall_score(y_tfidf_test, y_pred_test))
print("Testing F1 Score: ", f1_score(y_tfidf_test, y_pred_test))

Statistics of SVM model trained using TF_IDF features: 
Testing Accuracy:  0.8721
Testing Precision:  0.8736012845601887
Testing Recall:  0.8701954315989404
Testing F1 Score:  0.8718950320512822


As mentioned above the SVM, a more complex model than the perceptron, produces much better results for the google word2vec features as well as the amazon reviews word2vec features. The TF_IDF features also produce good results which are in fact better than those produced by the word2vec features. From these results we can infer that, the SVM is better at capturing the complex dependencies between the word vectors and the corresponding labels, the gap in the performance between TF_IDF features model and the word2vec features model shows that the SVM isn't complex enough. We can also note that the SVM trained with our word2vec model features still outperforms the one trained with google word2vec model feature.

### Feed Forward Neural Networks

Preparing data for the ternary classification.

In [108]:
x_google_ternary = google_df.drop(['label'], axis = 1).values
y_google_ternary = google_df.label.values
x_amazon_ternary = amazon_df.drop(['label'], axis = 1).values
y_amazon_ternary = amazon_df.label.values

Train_Test Split for ternary classification dataset. 80/20 split.

In [109]:
x_google_ternary_train, x_google_ternary_test, y_google_ternary_train, y_google_ternary_test = train_test_split(x_google_ternary, y_google_ternary, test_size = 0.2, random_state = 42)
x_amazon_ternary_train, x_amazon_ternary_test, y_amazon_ternary_train, y_amazon_ternary_test = train_test_split(x_amazon_ternary, y_amazon_ternary, test_size = 0.2, random_state = 42)

In [2]:
import torch
import torch.nn.functional as func
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
# import pynvml
# def get_memory_free_MiB(gpu_index):
#     pynvml.nvmlInit()
#     handle = pynvml.nvmlDeviceGetHandleByIndex(int(gpu_index))
#     mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
#     return mem_info.free // 1024 ** 2
# get_memory_free_MiB(0)

Creating validation set. In total, 80/20/20 split for training, validation and testing sets.

In [112]:
x_google_train, x_google_val, y_google_train, y_google_val = train_test_split(x_google_train, y_google_train, test_size = 0.25, random_state = 42)
x_amazon_train, x_amazon_val, y_amazon_train, y_amazon_val = train_test_split(x_amazon_train, y_amazon_train, test_size = 0.25, random_state = 42)

x_google_ternary_train, x_google_ternary_val, y_google_ternary_train, y_google_ternary_val = train_test_split(x_google_ternary_train, y_google_ternary_train, test_size = 0.25, random_state = 42)
x_amazon_ternary_train, x_amazon_ternary_val, y_amazon_ternary_train, y_amazon_ternary_val = train_test_split(x_amazon_ternary_train, y_amazon_ternary_train, test_size = 0.25, random_state = 42)

Changing the numpy n-dimensional arrays to torch float tensors and torch long tensors and assigning them to "device" which would be the GPU (cuda core) if it is available, else it will be the CPU.

In [113]:
x_google_train, x_google_val, x_google_test, y_google_train, y_google_val, y_google_test = torch.FloatTensor(x_google_train).to(device), torch.FloatTensor(x_google_val).to(device), torch.FloatTensor(x_google_test).to(device), torch.LongTensor(y_google_train).to(device), torch.LongTensor(y_google_val).to(device), torch.LongTensor(y_google_test).to(device)
x_amazon_train, x_amazon_val, x_amazon_test, y_amazon_train, y_amazon_val, y_amazon_test = torch.FloatTensor(x_amazon_train), torch.FloatTensor(x_amazon_val).to(device), torch.FloatTensor(x_amazon_test).to(device), torch.LongTensor(y_amazon_train).to(device), torch.LongTensor(y_amazon_val).to(device), torch.LongTensor(y_amazon_test).to(device)
x_google_ternary_train, x_google_ternary_val, x_google_ternary_test, y_google_ternary_train, y_google_ternary_val, y_google_ternary_test = torch.FloatTensor(x_google_ternary_train).to(device), torch.FloatTensor(x_google_ternary_val).to(device), torch.FloatTensor(x_google_ternary_test).to(device), torch.LongTensor(y_google_ternary_train).to(device), torch.LongTensor(y_google_ternary_val).to(device), torch.LongTensor(y_google_ternary_test).to(device)
x_amazon_ternary_train, x_amazon_ternary_val, x_amazon_ternary_test, y_amazon_ternary_train, y_amazon_ternary_val, y_amazon_ternary_test = torch.FloatTensor(x_amazon_ternary_train).to(device), torch.FloatTensor(x_amazon_ternary_val).to(device), torch.FloatTensor(x_amazon_ternary_test).to(device), torch.LongTensor(y_amazon_ternary_train).to(device), torch.LongTensor(y_amazon_ternary_val).to(device), torch.LongTensor(y_amazon_ternary_test).to(device)

Creating the dataset class for our dataset that will be used by the DataLoader function later.

In [3]:
class Dataset(object):
    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

class data(Dataset):
    def __init__(self, inputs, transform = None):
        self.data = inputs
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        inputs = self.data[index][0]
        label = self.data[index][1]
        if self.transform is not None:
            inputs = self.transform(inputs)
            
        return inputs, label

Combining the features and labels into one tensor dataset and then changing their type to be the type of the dataset class that we have defined above.

In [115]:
batch_size = 20

google_train_dataset = TensorDataset(x_google_train, y_google_train)
google_train_dataset = data(google_train_dataset)
google_val_dataset = TensorDataset(x_google_val, y_google_val)
google_val_dataset = data(google_val_dataset)
google_test_dataset = TensorDataset(x_google_test, y_google_test)
google_test_dataset = data(google_test_dataset)

google_train_loader = DataLoader(google_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
google_val_loader = DataLoader(google_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
google_test_loader = DataLoader(google_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

google_ternary_train_dataset = TensorDataset(x_google_ternary_train, y_google_ternary_train)
google_ternary_train_dataset = data(google_ternary_train_dataset)
google_ternary_val_dataset = TensorDataset(x_google_ternary_val, y_google_ternary_val)
google_ternary_val_dataset = data(google_ternary_val_dataset)
google_ternary_test_dataset = TensorDataset(x_google_ternary_test, y_google_ternary_test)
google_ternary_test_dataset = data(google_ternary_test_dataset)

google_ternary_train_loader = DataLoader(google_ternary_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
google_ternary_val_loader = DataLoader(google_ternary_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
google_ternary_test_loader = DataLoader(google_ternary_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

amazon_train_dataset = TensorDataset(x_amazon_train, y_amazon_train)
amazon_train_dataset = data(amazon_train_dataset)
amazon_val_dataset = TensorDataset(x_amazon_val, y_amazon_val)
amazon_val_dataset = data(amazon_val_dataset)
amazon_test_dataset = TensorDataset(x_amazon_test, y_amazon_test)
amazon_test_dataset = data(amazon_test_dataset)

amazon_train_loader = DataLoader(amazon_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
amazon_val_loader = DataLoader(amazon_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
amazon_test_loader = DataLoader(amazon_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

amazon_ternary_train_dataset = TensorDataset(x_amazon_ternary_train, y_amazon_ternary_train)
amazon_ternary_train_dataset = data(amazon_ternary_train_dataset)
amazon_ternary_val_dataset = TensorDataset(x_amazon_ternary_val, y_amazon_ternary_val)
amazon_ternary_val_dataset = data(amazon_ternary_val_dataset)
amazon_ternary_test_dataset = TensorDataset(x_amazon_ternary_test, y_amazon_ternary_test)
amazon_ternary_test_dataset = data(amazon_ternary_test_dataset)

amazon_ternary_train_loader = DataLoader(amazon_ternary_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
amazon_ternary_val_loader = DataLoader(amazon_ternary_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
amazon_ternary_test_loader = DataLoader(amazon_ternary_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

Initializing the feed forward neural network model for the binary classification task. Hyperparameters, learning_rate = 0.03, dropout = 0.2 and SGD optimizer have been used across all the average vector Feed-Forward network models.

In [116]:
class ff_nn_binary(torch.nn.Module):
    def __init__(self):
        super(ff_nn_binary, self).__init__()
        hidden_1 = 50
        hidden_2 = 10
        self.fc1 = torch.nn.Linear(300, hidden_1)
        self.fc2 = torch.nn.Linear(hidden_1, hidden_2)
        self.fc3 = torch.nn.Linear(hidden_2, 1)
        self.dropout = torch.nn.Dropout(0.2)
    
    def forward(self, x):
        x = func.relu(self.fc1(x))
        x = self.dropout(x)
        x = func.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

model = ff_nn_binary().to(device)
print(model)

ff_nn_binary(
  (fc1): Linear(in_features=300, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


Initializing the SGD optimizer with the model parameters also initialising the Binary Cross Entropy loss function.

In [117]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.03)

Training the neural network model using the google word2vec features.

In [118]:
epochs = 20
valid_min_loss = np.inf
train_loader = google_train_loader
val_loader = google_val_loader
test_loader = google_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs.to(device), target.to(device)
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/google_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.034653 	Validation Loss: 0.034625
Validation loss decreased (inf --> 0.034625). Saving model...
Epoch: 2 	Training Loss: 0.034518 	Validation Loss: 0.034131
Validation loss decreased (0.034625 --> 0.034131). Saving model...
Epoch: 3 	Training Loss: 0.031292 	Validation Loss: 0.025858
Validation loss decreased (0.034131 --> 0.025858). Saving model...
Epoch: 4 	Training Loss: 0.026738 	Validation Loss: 0.023430
Validation loss decreased (0.025858 --> 0.023430). Saving model...
Epoch: 5 	Training Loss: 0.025417 	Validation Loss: 0.022585
Validation loss decreased (0.023430 --> 0.022585). Saving model...
Epoch: 6 	Training Loss: 0.024582 	Validation Loss: 0.023827
Epoch: 7 	Training Loss: 0.024073 	Validation Loss: 0.022912
Epoch: 8 	Training Loss: 0.023661 	Validation Loss: 0.021227
Validation loss decreased (0.022585 --> 0.021227). Saving model...
Epoch: 9 	Training Loss: 0.023488 	Validation Loss: 0.023560
Epoch: 10 	Training Loss: 0.023277 	Validation Loss: 0

Testing the model on the test dataset. And as you can see, the google model's accuracy has gone up indicating that the feed-forward neural network is indeed a more complex model than perceptron and SVM and is able to capture the underlying relationship between the average word vector and the label better. The accuracy is still isn't vastly better, and this may be due to two reasons. One, the model still isn't complex enough or two, the svm and perceptron train for 1000 iterations (maximum) and the FFNN (feed-forward neural network) just trains for 20 epochs thus limiting the model's performance.

In [119]:
y_pred_list = []
y_targ_list = []
test_loader = google_test_loader
model = ff_nn_binary().to(device)
model.load_state_dict(torch.load('model/google_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        y_test_pred = model(inputs)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Google Word2Vec features for binary classification model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Google Word2Vec features for binary classification model
              precision    recall  f1-score   support

           0       0.79      0.87      0.82     19993
           1       0.85      0.76      0.81     20007

    accuracy                           0.82     40000
   macro avg       0.82      0.82      0.82     40000
weighted avg       0.82      0.82      0.82     40000



Re-initialising the model and the optimizer so that the weights, gradients and other model parameters aren't carried over.

In [124]:
del model
del optimizer
del loss_fn
model = ff_nn_binary().to(device)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.03)

Training the neural network model using the amazon review word2vec features.

In [125]:
epochs = 20

valid_min_loss = np.inf
train_loader = amazon_train_loader
val_loader = amazon_val_loader
test_loader = amazon_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/amazon_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.024246 	Validation Loss: 0.019032
Validation loss decreased (inf --> 0.019032). Saving model...
Epoch: 2 	Training Loss: 0.019844 	Validation Loss: 0.018640
Validation loss decreased (0.019032 --> 0.018640). Saving model...
Epoch: 3 	Training Loss: 0.019228 	Validation Loss: 0.017923
Validation loss decreased (0.018640 --> 0.017923). Saving model...
Epoch: 4 	Training Loss: 0.018770 	Validation Loss: 0.017490
Validation loss decreased (0.017923 --> 0.017490). Saving model...
Epoch: 5 	Training Loss: 0.018378 	Validation Loss: 0.017336
Validation loss decreased (0.017490 --> 0.017336). Saving model...
Epoch: 6 	Training Loss: 0.018105 	Validation Loss: 0.017187
Validation loss decreased (0.017336 --> 0.017187). Saving model...
Epoch: 7 	Training Loss: 0.017803 	Validation Loss: 0.017401
Epoch: 8 	Training Loss: 0.017596 	Validation Loss: 0.016839
Validation loss decreased (0.017187 --> 0.016839). Saving model...
Epoch: 9 	Training Loss: 0.017475 	Validation Lo

Testing the model, and once again as you can see from below, the amazon word2vec feature model outperforms its counterpart. And, as noted in the google word2vec feature model, the FFNN for amazon word2vec features perform better than the simple models (SVM, Perceptron).

In [126]:
y_pred_list = []
y_targ_list = []
test_loader = amazon_test_loader
model = ff_nn_binary().to(device)
model.load_state_dict(torch.load('model/amazon_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        y_test_pred = model(inputs)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Amazon Word2Vec features for binary classification model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Amazon Word2Vec features for binary classification model
              precision    recall  f1-score   support

           0       0.86      0.85      0.86     19892
           1       0.86      0.86      0.86     20088

    accuracy                           0.86     39980
   macro avg       0.86      0.86      0.86     39980
weighted avg       0.86      0.86      0.86     39980



In [127]:
class ff_nn_ternary(torch.nn.Module):
    def __init__(self):
        super(ff_nn_ternary, self).__init__()
        hidden_1 = 50
        hidden_2 = 10
        self.fc1 = torch.nn.Linear(300, hidden_1)
        self.fc2 = torch.nn.Linear(hidden_1, hidden_2)
        self.fc3 = torch.nn.Linear(hidden_2, 3)
        self.dropout = torch.nn.Dropout(0.2)
    
    def forward(self, x):
        x = func.relu(self.fc1(x))
        x = self.dropout(x)
        x = func.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

model = ff_nn_ternary()
print(model)

ff_nn_ternary(
  (fc1): Linear(in_features=300, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


Since the dataset is skewed towards two classes (positive ~100k samples and negative ~100k samples, neutral ~50k samples), we have to include class weights so that the model could be penalized more for making a mistake in predicting the smaller sized class which ultimately leads to better performance. 

In [128]:
np_ary = y_google_ternary
class_weights_google = class_weight.compute_class_weight('balanced', np.unique(np_ary), np_ary)
print("Class weights of ternary classification for Google word2vec feature model:")
print(class_weights_google)
np_ary = y_amazon_ternary
class_weights_amazon = class_weight.compute_class_weight('balanced', np.unique(np_ary), np_ary)
print("\nClass weights of ternary classification for Amazon word2vec feature model:")
print(class_weights_amazon)

Class weights of ternary classification for Google word2vec feature model:
[0.83333333 0.83333333 1.66666667]

Class weights of ternary classification for Amazon word2vec feature model:
[0.83329833 0.83333167 1.66681336]


In [131]:
del model
del optimizer
del loss_fn

In [132]:
model = ff_nn_ternary().to(device)
class_weights = torch.FloatTensor(class_weights_google)
loss_fn = torch.nn.CrossEntropyLoss(weight = class_weights).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.03)

In [133]:
epochs = 20

valid_min_loss = np.inf
train_loader = google_ternary_train_loader
val_loader = google_ternary_val_loader
test_loader = google_ternary_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs.to(device), target.to(device)
        output = model(inputs)
        loss = loss_fn(output, target)
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss, valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/google_ternary_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.054932 	Validation Loss: 0.054913
Validation loss decreased (inf --> 0.054913). Saving model...
Epoch: 2 	Training Loss: 0.054841 	Validation Loss: 0.054534
Validation loss decreased (0.054913 --> 0.054534). Saving model...
Epoch: 3 	Training Loss: 0.052012 	Validation Loss: 0.047934
Validation loss decreased (0.054534 --> 0.047934). Saving model...
Epoch: 4 	Training Loss: 0.048529 	Validation Loss: 0.045973
Validation loss decreased (0.047934 --> 0.045973). Saving model...
Epoch: 5 	Training Loss: 0.047422 	Validation Loss: 0.045821
Validation loss decreased (0.045973 --> 0.045821). Saving model...
Epoch: 6 	Training Loss: 0.046736 	Validation Loss: 0.044438
Validation loss decreased (0.045821 --> 0.044438). Saving model...
Epoch: 7 	Training Loss: 0.046318 	Validation Loss: 0.045213
Epoch: 8 	Training Loss: 0.045980 	Validation Loss: 0.043992
Validation loss decreased (0.044438 --> 0.043992). Saving model...
Epoch: 9 	Training Loss: 0.045614 	Validation Lo

As you can see from the below statistics the model performs significantly worse on ternary classification when compared to the binary classification task. This again, could be due to two reasons. One, the dataset isn't proper. Specifically, the neutral class has to be pretty ambiguous to produce such results. By ambiguous, I mean that the neutral reviews aren't totally neutral and they are either somewhat positive or somewhat negative. The ambiguity of the neutral class affects the precision and recall of both the negative and positive classes thus bringing the whole performance of the model down. Two, the relation between the vectors and the labels with the addition of the neutral class has become more complex such that the model is no longer able to represent that relation effectively. This could be further corroborated by the training loss which is pretty high even after 20 epochs.

In [134]:
y_pred_list = []
y_targ_list = []
test_loader = google_ternary_test_loader
model = ff_nn_ternary().to(device)
model.load_state_dict(torch.load('model/google_ternary_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Google Word2Vec features for ternary classification model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Google Word2Vec features for ternary classification model
              precision    recall  f1-score   support

           0       0.73      0.68      0.70     19850
           1       0.76      0.68      0.72     20114
           2       0.35      0.46      0.40     10036

    accuracy                           0.64     50000
   macro avg       0.61      0.61      0.61     50000
weighted avg       0.66      0.64      0.65     50000



In [137]:
del model
del optimizer
del loss_fn

In [138]:
model = ff_nn_ternary().to(device)
class_weights = torch.FloatTensor(class_weights_amazon)
loss_fn = torch.nn.CrossEntropyLoss(weight = class_weights).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.03)

In [139]:
epochs = 20

valid_min_loss = np.inf
train_loader = amazon_ternary_train_loader
val_loader = amazon_ternary_val_loader
test_loader = amazon_ternary_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
#         target = target.squeeze(1)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs.to(device), target.to(device)
        output = model(inputs)
#         target = target.squeeze(1)
        loss = loss_fn(output, target)
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss, valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/amazon_ternary_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.047069 	Validation Loss: 0.041461
Validation loss decreased (inf --> 0.041461). Saving model...
Epoch: 2 	Training Loss: 0.042255 	Validation Loss: 0.040320
Validation loss decreased (0.041461 --> 0.040320). Saving model...
Epoch: 3 	Training Loss: 0.041527 	Validation Loss: 0.039922
Validation loss decreased (0.040320 --> 0.039922). Saving model...
Epoch: 4 	Training Loss: 0.041095 	Validation Loss: 0.039999
Epoch: 5 	Training Loss: 0.040745 	Validation Loss: 0.039532
Validation loss decreased (0.039922 --> 0.039532). Saving model...
Epoch: 6 	Training Loss: 0.040532 	Validation Loss: 0.039621
Epoch: 7 	Training Loss: 0.040253 	Validation Loss: 0.039001
Validation loss decreased (0.039532 --> 0.039001). Saving model...
Epoch: 8 	Training Loss: 0.040066 	Validation Loss: 0.039078
Epoch: 9 	Training Loss: 0.039973 	Validation Loss: 0.038908
Validation loss decreased (0.039001 --> 0.038908). Saving model...
Epoch: 10 	Training Loss: 0.039891 	Validation Loss: 0

Once again, the amazon word2vec feature model produces better results than the google word2vec feature model. Still, the preformance is pretty mediocre in comparison to the binary classification performance.

In [140]:
y_pred_list = []
y_targ_list = []
test_loader = amazon_ternary_test_loader
model = ff_nn_ternary().to(device)
model.load_state_dict(torch.load('model/amazon_ternary_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Amazon Word2Vec features for ternary classification model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Google Word2Vec features for ternary classification model
              precision    recall  f1-score   support

           0       0.77      0.69      0.73     19834
           1       0.80      0.72      0.76     20219
           2       0.37      0.54      0.44      9927

    accuracy                           0.67     49980
   macro avg       0.65      0.65      0.64     49980
weighted avg       0.71      0.67      0.68     49980



## FFNN with word vectors of first 10 words of the review

Below is the function that calculates the vector representations of the first 10 words of the review and returns a np.ndarray. We ignore words that aren't present in a model and if a review doesn't have any words or if none of the words in the review are present in the specified model, we return a nan value which would be removed later.

In [10]:
def compute_10_w2v(sentence, model):
    result_vector = np.zeros(3000, dtype = np.float64)
    count = 0
    split_list = sentence.split(" ")
    len_list = len(split_list)
    if model == 'google':
        for word in split_list:
            if count == 10 or count == len_list:
                break
            count += 1
            try:
                result_vector[(count - 1) * 300 : count * 300] = w2v_google[word]
            except KeyError:
                count -= 1
                continue
    else:
        for word in split_list:
            if count == 10 or count == len_list:
                break
            count += 1
            try:
                result_vector[(count - 1) * 300 : count * 300] = w2v_model.wv[word]
            except KeyError:
                count -= 1
                continue
    
    if np.all((result_vector == 0)):    
        return np.nan
    return result_vector

We store the vector and the label as a list of tuples, as we previously did, and then we remove nan values. Later, we store it as a pickle file (reading and writing a pickle file for such a big dataset proved to be much faster when compared to csv file).

In [13]:
google_f10 = [(compute_10_w2v(df_new.iloc[[x]].review_body.values[0], "google"), df_new.iloc[[x]].star_rating.values[0]) for x in range(len(df_new))]
google_f10 = [x for x in google_f10 if np.isnan(x[0]).any() == False]
with open("google_f10.pkl", "wb") as file_obj:
    pkl.dump(google_f10, file_obj)
del google_f10

In [14]:
amazon_f10 = [(compute_10_w2v(df_new.iloc[[x]].review_body.values[0], "google"), df_new.iloc[[x]].star_rating.values[0]) for x in range(len(df_new))]
amazon_f10 = [x for x in amazon_f10 if np.isnan(x[0]).any() == False]
with open("amazon_f10.pkl", "wb") as file_obj:
    pkl.dump(amazon_f10, file_obj)
del amazon_f10

##### Checkpoint - 4

In [2]:
with open("data/google_f10.pkl", "rb") as file_obj:
    google_f10 = pkl.load(file_obj)
    
with open("data/amazon_f10.pkl", "rb") as file_obj:
    amazon_f10 = pkl.load(file_obj)

Now, we create the binary dataset from the original dataset and then separate them into input(x) and output(y) for both binary and ternary classification tasks.

In [16]:
google_f10_binary = [x for x in google_f10 if x[1] == 0 or x[1] == 1]
amazon_f10_binary = [x for x in amazon_f10 if x[1] == 0 or x[1] == 1]

In [17]:
x_google_f10 = np.array([x[0] for x in google_f10_binary])
y_google_f10 = np.array([x[1] for x in google_f10_binary])

x_amazon_f10 = np.array([x[0] for x in amazon_f10_binary])
y_amazon_f10 = np.array([x[1] for x in amazon_f10_binary])

In [18]:
x_google_f10_ternary = np.array([x[0] for x in google_f10])
y_google_f10_ternary = np.array([x[1] for x in google_f10])

x_amazon_f10_ternary = np.array([x[0] for x in amazon_f10])
y_amazon_f10_ternary = np.array([x[1] for x in amazon_f10])

Then, we split the data into train and test. 80/20 split.

In [32]:
x_google_f10_train, x_google_f10_test, y_google_f10_train, y_google_f10_test = train_test_split(x_google_f10, y_google_f10, test_size=0.2, random_state=42)
x_amazon_f10_train, x_amazon_f10_test, y_amazon_f10_train, y_amazon_f10_test = train_test_split(x_amazon_f10, y_amazon_f10, test_size=0.2, random_state=42)

In [33]:
x_google_f10_ternary_train, x_google_f10_ternary_test, y_google_f10_ternary_train, y_google_f10_ternary_test = train_test_split(x_google_f10_ternary, y_google_f10_ternary, test_size = 0.2, random_state = 42)
x_amazon_f10_ternary_train, x_amazon_f10_ternary_test, y_amazon_f10_ternary_train, y_amazon_f10_ternary_test = train_test_split(x_amazon_f10_ternary, y_amazon_f10_ternary, test_size = 0.2, random_state = 42)

80/20/20 - train/val/test split.

In [34]:
x_google_f10_train, x_google_f10_val, y_google_f10_train, y_google_f10_val = train_test_split(x_google_f10_train, y_google_f10_train, test_size = 0.25, random_state = 42)
x_amazon_f10_train, x_amazon_f10_val, y_amazon_f10_train, y_amazon_f10_val = train_test_split(x_amazon_f10_train, y_amazon_f10_train, test_size = 0.25, random_state = 42)

x_google_f10_ternary_train, x_google_f10_ternary_val, y_google_f10_ternary_train, y_google_f10_ternary_val = train_test_split(x_google_f10_ternary_train, y_google_f10_ternary_train, test_size = 0.25, random_state = 42)
x_amazon_f10_ternary_train, x_amazon_f10_ternary_val, y_amazon_f10_ternary_train, y_amazon_f10_ternary_val = train_test_split(x_amazon_f10_ternary_train, y_amazon_f10_ternary_train, test_size = 0.25, random_state = 42)

Converting the arrays into float and long tensors.

In [35]:
x_google_f10_train, x_google_f10_val, x_google_f10_test, y_google_f10_train, y_google_f10_val, y_google_f10_test = torch.FloatTensor(x_google_f10_train), torch.FloatTensor(x_google_f10_val), torch.FloatTensor(x_google_f10_test), torch.LongTensor(y_google_f10_train), torch.LongTensor(y_google_f10_val), torch.LongTensor(y_google_f10_test)
x_amazon_f10_train, x_amazon_f10_val, x_amazon_f10_test, y_amazon_f10_train, y_amazon_f10_val, y_amazon_f10_test = torch.FloatTensor(x_amazon_f10_train), torch.FloatTensor(x_amazon_f10_val), torch.FloatTensor(x_amazon_f10_test), torch.LongTensor(y_amazon_f10_train), torch.LongTensor(y_amazon_f10_val), torch.LongTensor(y_amazon_f10_test)
x_google_f10_ternary_train, x_google_f10_ternary_val, x_google_f10_ternary_test, y_google_f10_ternary_train, y_google_f10_ternary_val, y_google_f10_ternary_test = torch.FloatTensor(x_google_f10_ternary_train), torch.FloatTensor(x_google_f10_ternary_val), torch.FloatTensor(x_google_f10_ternary_test), torch.LongTensor(y_google_f10_ternary_train), torch.LongTensor(y_google_f10_ternary_val), torch.LongTensor(y_google_f10_ternary_test)
x_amazon_f10_ternary_train, x_amazon_f10_ternary_val, x_amazon_f10_ternary_test, y_amazon_f10_ternary_train, y_amazon_f10_ternary_val, y_amazon_f10_ternary_test = torch.FloatTensor(x_amazon_f10_ternary_train), torch.FloatTensor(x_amazon_f10_ternary_val), torch.FloatTensor(x_amazon_f10_ternary_test), torch.LongTensor(y_amazon_f10_ternary_train), torch.LongTensor(y_amazon_f10_ternary_val), torch.LongTensor(y_amazon_f10_ternary_test)

Initializing the data loaders for different datasets.

In [38]:
batch_size = 20

google_f10_train_dataset = TensorDataset(x_google_f10_train, y_google_f10_train)
google_f10_train_dataset = data(google_f10_train_dataset)
google_f10_val_dataset = TensorDataset(x_google_f10_val, y_google_f10_val)
google_f10_val_dataset = data(google_f10_val_dataset)
google_f10_test_dataset = TensorDataset(x_google_f10_test, y_google_f10_test)
google_f10_test_dataset = data(google_f10_test_dataset)

google_f10_train_loader = DataLoader(google_f10_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
google_f10_val_loader = DataLoader(google_f10_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
google_f10_test_loader = DataLoader(google_f10_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

google_f10_ternary_train_dataset = TensorDataset(x_google_f10_ternary_train, y_google_f10_ternary_train)
google_f10_ternary_train_dataset = data(google_f10_ternary_train_dataset)
google_f10_ternary_val_dataset = TensorDataset(x_google_f10_ternary_val, y_google_f10_ternary_val)
google_f10_ternary_val_dataset = data(google_f10_ternary_val_dataset)
google_f10_ternary_test_dataset = TensorDataset(x_google_f10_ternary_test, y_google_f10_ternary_test)
google_f10_ternary_test_dataset = data(google_f10_ternary_test_dataset)

google_f10_ternary_train_loader = DataLoader(google_f10_ternary_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
google_f10_ternary_val_loader = DataLoader(google_f10_ternary_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
google_f10_ternary_test_loader = DataLoader(google_f10_ternary_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

amazon_f10_train_dataset = TensorDataset(x_amazon_f10_train, y_amazon_f10_train)
amazon_f10_train_dataset = data(amazon_f10_train_dataset)
amazon_f10_val_dataset = TensorDataset(x_amazon_f10_val, y_amazon_f10_val)
amazon_f10_val_dataset = data(amazon_f10_val_dataset)
amazon_f10_test_dataset = TensorDataset(x_amazon_f10_test, y_amazon_f10_test)
amazon_f10_test_dataset = data(amazon_f10_test_dataset)

amazon_f10_train_loader = DataLoader(amazon_f10_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
amazon_f10_val_loader = DataLoader(amazon_f10_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
amazon_f10_test_loader = DataLoader(amazon_f10_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

amazon_f10_ternary_train_dataset = TensorDataset(x_amazon_f10_ternary_train, y_amazon_f10_ternary_train)
amazon_f10_ternary_train_dataset = data(amazon_f10_ternary_train_dataset)
amazon_f10_ternary_val_dataset = TensorDataset(x_amazon_f10_ternary_val, y_amazon_f10_ternary_val)
amazon_f10_ternary_val_dataset = data(amazon_f10_ternary_val_dataset)
amazon_f10_ternary_test_dataset = TensorDataset(x_amazon_f10_ternary_test, y_amazon_f10_ternary_test)
amazon_f10_ternary_test_dataset = data(amazon_f10_ternary_test_dataset)

amazon_f10_ternary_train_loader = DataLoader(amazon_f10_ternary_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
amazon_f10_ternary_val_loader = DataLoader(amazon_f10_ternary_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
amazon_f10_ternary_test_loader = DataLoader(amazon_f10_ternary_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

Declaring the class for the binary classification network. Hyperparameters: learning rate - 0.03, dropout - 0.25 and SGD optimizer for both google and amazon word2vec feature models. Keeping the epochs at 10 as the models tend to overfit the data after 10 epochs and thus no improvement in validation loss is seen after that point.

In [93]:
class ff_nn_binary_f10(torch.nn.Module):
    def __init__(self):
        super(ff_nn_binary_f10, self).__init__()
        hidden_1 = 50
        hidden_2 = 10
        self.fc1 = torch.nn.Linear(3000, hidden_1)
        self.fc2 = torch.nn.Linear(hidden_1, hidden_2)
        self.fc3 = torch.nn.Linear(hidden_2, 1)
        self.dropout = torch.nn.Dropout(0.25)
    
    def forward(self, x):
        x = func.relu(self.fc1(x))
        x = self.dropout(x)
        x = func.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

model = ff_nn_binary_f10().to(device)
print(model)

ff_nn_binary_f10(
  (fc1): Linear(in_features=3000, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [94]:
del model
del optimizer
del loss_fn

In [95]:
model = ff_nn_binary_f10().to(device)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.03)

In [96]:
epochs = 10
valid_min_loss = np.inf
train_loader = google_f10_train_loader
val_loader = google_f10_val_loader
test_loader = google_f10_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs.to(device), target.to(device)
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/google_f10_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.027619 	Validation Loss: 0.024006
Validation loss decreased (inf --> 0.024006). Saving model...
Epoch: 2 	Training Loss: 0.024168 	Validation Loss: 0.023455
Validation loss decreased (0.024006 --> 0.023455). Saving model...
Epoch: 3 	Training Loss: 0.023029 	Validation Loss: 0.023116
Validation loss decreased (0.023455 --> 0.023116). Saving model...
Epoch: 4 	Training Loss: 0.021966 	Validation Loss: 0.023071
Validation loss decreased (0.023116 --> 0.023071). Saving model...
Epoch: 5 	Training Loss: 0.020911 	Validation Loss: 0.023256
Epoch: 6 	Training Loss: 0.019832 	Validation Loss: 0.023382
Epoch: 7 	Training Loss: 0.018757 	Validation Loss: 0.023968
Epoch: 8 	Training Loss: 0.017801 	Validation Loss: 0.024939
Epoch: 9 	Training Loss: 0.016821 	Validation Loss: 0.025995
Epoch: 10 	Training Loss: 0.016007 	Validation Loss: 0.026265


After viewing the results below, it must be surprising to see that the average vector model managed to outperform the first 10 words vector model. Logically, the latter should be able to capture the underlying relation between the word vectors and the labels better. But that is not the case, why? The answer is pretty straight-forward. There are 3000 input features and just ~60k samples per label. This proportion (no.of datapoints / no.of features) is clearly very small and thus reduces the performance. One solution would be to increase the no.of datapoints. Another would be to use a much more complex model that could fit with just the given no.of datapoints.

In [97]:
y_pred_list = []
y_targ_list = []
test_loader = google_f10_test_loader
model = ff_nn_binary_f10().to(device)
model.load_state_dict(torch.load('model/google_f10_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        y_test_pred = model(inputs)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Google Word2Vec features of first 10 words for binary classification model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Google Word2Vec features of first 10 words for binary classification model
              precision    recall  f1-score   support

           0       0.78      0.78      0.78     19994
           1       0.78      0.78      0.78     19946

    accuracy                           0.78     39940
   macro avg       0.78      0.78      0.78     39940
weighted avg       0.78      0.78      0.78     39940



In [102]:
del model
del optimizer
del loss_fn

In [103]:
model = ff_nn_binary_f10().to(device)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.03)

In [104]:
epochs = 10
valid_min_loss = np.inf
train_loader = amazon_f10_train_loader
val_loader = amazon_f10_val_loader
test_loader = amazon_f10_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs.to(device), target.to(device)
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/amazon_f10_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.025965 	Validation Loss: 0.023710
Validation loss decreased (inf --> 0.023710). Saving model...
Epoch: 2 	Training Loss: 0.023627 	Validation Loss: 0.023347
Validation loss decreased (0.023710 --> 0.023347). Saving model...
Epoch: 3 	Training Loss: 0.022361 	Validation Loss: 0.023157
Validation loss decreased (0.023347 --> 0.023157). Saving model...
Epoch: 4 	Training Loss: 0.021295 	Validation Loss: 0.023167
Epoch: 5 	Training Loss: 0.020079 	Validation Loss: 0.023377
Epoch: 6 	Training Loss: 0.019068 	Validation Loss: 0.023652
Epoch: 7 	Training Loss: 0.018117 	Validation Loss: 0.024583
Epoch: 8 	Training Loss: 0.017081 	Validation Loss: 0.025771
Epoch: 9 	Training Loss: 0.016279 	Validation Loss: 0.025938
Epoch: 10 	Training Loss: 0.015382 	Validation Loss: 0.026504


Still, the amazon word2vec feature model outperforms the google model.

In [105]:
y_pred_list = []
y_targ_list = []
test_loader = amazon_f10_test_loader
model = ff_nn_binary_f10().to(device)
model.load_state_dict(torch.load('model/amazon_f10_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        y_test_pred = model(inputs)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Amazon Word2Vec features of first 10 words for binary classification model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Amazon Word2Vec features of first 10 words for binary classification model
              precision    recall  f1-score   support

           0       0.76      0.81      0.78     19994
           1       0.79      0.74      0.77     19946

    accuracy                           0.78     39940
   macro avg       0.78      0.78      0.78     39940
weighted avg       0.78      0.78      0.78     39940



Declaring the class for the ternary classification network model. Hyperparameters: learning rate - 0.001, dropout - 0.2 and optimizer - Adam. The hyperparameters are same for both the google and amazon word2vec feature models. Epochs set to be 10.

In [112]:
class ff_nn_ternary_f10(torch.nn.Module):
    def __init__(self):
        super(ff_nn_ternary_f10, self).__init__()
        hidden_1 = 50
        hidden_2 = 10
        self.fc1 = torch.nn.Linear(3000, hidden_1)
        self.fc2 = torch.nn.Linear(hidden_1, hidden_2)
        self.fc3 = torch.nn.Linear(hidden_2, 3)
        self.dropout = torch.nn.Dropout(0.2)
    
    def forward(self, x):
        x = func.relu(self.fc1(x))
        x = self.dropout(x)
        x = func.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

model = ff_nn_ternary_f10()
print(model)

ff_nn_ternary_f10(
  (fc1): Linear(in_features=3000, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [113]:
np_ary = y_google_f10_ternary
class_weights_google_f10 = class_weight.compute_class_weight('balanced', np.unique(np_ary), np_ary)
print("Class weights of ternary classification for Google word2vec feature model:")
print(class_weights_google_f10)
np_ary = y_amazon_f10_ternary
class_weights_amazon_f10 = class_weight.compute_class_weight('balanced', np.unique(np_ary), np_ary)
print("\nClass weights of ternary classification for Amazon word2vec feature model:")
print(class_weights_amazon_f10)

Class weights of ternary classification for Google word2vec feature model:
[0.83287476 0.83376736 1.66676678]

Class weights of ternary classification for Amazon word2vec feature model:
[0.83287476 0.83376736 1.66676678]


In [132]:
del model
del optimizer
del loss_fn

In [133]:
model = ff_nn_ternary_f10().to(device)
class_weights = torch.FloatTensor(class_weights_google_f10)
loss_fn = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [134]:
epochs = 10

valid_min_loss = np.inf
train_loader = google_f10_ternary_train_loader
val_loader = google_f10_ternary_val_loader
test_loader = google_f10_ternary_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs.to(device), target.to(device)
        output = model(inputs)
        loss = loss_fn(output, target)
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss, valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/google_f10_ternary_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.044378 	Validation Loss: 0.042580
Validation loss decreased (inf --> 0.042580). Saving model...
Epoch: 2 	Training Loss: 0.042055 	Validation Loss: 0.042112
Validation loss decreased (0.042580 --> 0.042112). Saving model...
Epoch: 3 	Training Loss: 0.040667 	Validation Loss: 0.041951
Validation loss decreased (0.042112 --> 0.041951). Saving model...
Epoch: 4 	Training Loss: 0.039281 	Validation Loss: 0.042345
Epoch: 5 	Training Loss: 0.038057 	Validation Loss: 0.043076
Epoch: 6 	Training Loss: 0.036943 	Validation Loss: 0.043428
Epoch: 7 	Training Loss: 0.035902 	Validation Loss: 0.043986
Epoch: 8 	Training Loss: 0.034921 	Validation Loss: 0.044366
Epoch: 9 	Training Loss: 0.034043 	Validation Loss: 0.045273
Epoch: 10 	Training Loss: 0.033275 	Validation Loss: 0.046074


As we saw in the average vector FFNN ternary classification models, the performance of this ternary classification model is also pretty mediocre. And the reasons for it remain the same as the ones given for the average vetor FFNN models.

In [135]:
y_pred_list = []
y_targ_list = []
test_loader = google_f10_ternary_test_loader
model = ff_nn_ternary_f10().to(device)
model.load_state_dict(torch.load('model/google_f10_ternary_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Google Word2Vec features of fist 10 words for ternary classification model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Google Word2Vec features of fist 10 words for ternary classification model
              precision    recall  f1-score   support

           0       0.62      0.76      0.68     19855
           1       0.65      0.76      0.70     20109
           2       0.47      0.10      0.16      9976

    accuracy                           0.63     49940
   macro avg       0.58      0.54      0.51     49940
weighted avg       0.60      0.63      0.58     49940



In [136]:
del model
del optimizer
del loss_fn

In [137]:
model = ff_nn_ternary_f10().to(device)
class_weights = torch.FloatTensor(class_weights_google_f10)
loss_fn = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [138]:
epochs = 10

valid_min_loss = np.inf
train_loader = amazon_f10_ternary_train_loader
val_loader = amazon_f10_ternary_val_loader
test_loader = amazon_f10_ternary_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs.to(device), target.to(device)
        output = model(inputs)
        loss = loss_fn(output, target)
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss, valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/amazon_f10_ternary_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.044314 	Validation Loss: 0.042600
Validation loss decreased (inf --> 0.042600). Saving model...
Epoch: 2 	Training Loss: 0.042040 	Validation Loss: 0.042173
Validation loss decreased (0.042600 --> 0.042173). Saving model...
Epoch: 3 	Training Loss: 0.040572 	Validation Loss: 0.042054
Validation loss decreased (0.042173 --> 0.042054). Saving model...
Epoch: 4 	Training Loss: 0.039175 	Validation Loss: 0.042477
Epoch: 5 	Training Loss: 0.037797 	Validation Loss: 0.042906
Epoch: 6 	Training Loss: 0.036613 	Validation Loss: 0.043617
Epoch: 7 	Training Loss: 0.035505 	Validation Loss: 0.044600
Epoch: 8 	Training Loss: 0.034552 	Validation Loss: 0.045405
Epoch: 9 	Training Loss: 0.033705 	Validation Loss: 0.046492
Epoch: 10 	Training Loss: 0.032878 	Validation Loss: 0.047008


In this case, the performances of the amazon and the google word2vec feature models are nearly the same. The only patent difference is seen in the F1 score of the neutral class. This shows that the amazon model manages to edge out the google model by a very thin margin.

In [139]:
y_pred_list = []
y_targ_list = []
test_loader = amazon_f10_ternary_test_loader
model = ff_nn_ternary_f10().to(device)
model.load_state_dict(torch.load('model/amazon_f10_ternary_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Amazon Word2Vec features of fist 10 words for ternary classification model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Amazon Word2Vec features of fist 10 words for ternary classification model
              precision    recall  f1-score   support

           0       0.62      0.77      0.68     19854
           1       0.66      0.73      0.69     20109
           2       0.45      0.12      0.19      9977

    accuracy                           0.63     49940
   macro avg       0.57      0.54      0.52     49940
weighted avg       0.60      0.63      0.59     49940



Conclusion: The neural network models manage to perform better than the simple models for the binary classification task. The ternary classification task performance is average for both the amazon and the google word2vec feature models indicating that either the dataset is not proper or that the model isn't complex enough.

## RNN model

Declaring the function that calculates the word2vec features for the first 50 words of a review and stores them in separate dimensions, indicating the 50 timesteps. Once again, we throw away reviews with no words or reviews where none of the words are modeled by either of the word2vec models. If the review length is less than 50 words, we pad the remaining spaces with zero vectors, and this is taken care by initializing the result array as a numpy zeros array.

In [ ]:
def compute_rnn_ip(sentence, model):
    split_list = sentence.split(" ")
    result_array = np.zeros((50, 300), dtype = np.float32)
    idx = 0
    if model == "google":
        for word in split_list:
            if idx == 50 or idx == len(split_list):
                break
            try:
                result_array[idx] = w2v_google[word]
                idx += 1
            except KeyError:
                continue
    else:
        for word in split_list:
            if idx == 50 or idx == len(split_list):
                break
            try:
                result_array[idx] = w2v_model.wv[word]
                idx += 1
            except KeyError:
                continue
    if idx == 0:
        return np.nan
    return result_array

As we did before, we create a list of tuples depicting the dataset and we drop the datapoints with nan values. Then we store the datasets as pickle files for easier and faster access in the future.

In [ ]:
rnn_google_df = [(compute_rnn_ip(df_new.iloc[[x]].review_body.values[0], "google"), df_new.iloc[[x]].star_rating.values[0]) for x in range(len(df_new))]
rnn_google_df = [x for x in rnn_google_df if np.isnan(x[0]).any() == False]

In [ ]:
with open("data/rnn_google.pkl", "wb") as file_obj:
    pkl.dump(rnn_google_df, file_obj)

del rnn_google_df

In [ ]:
rnn_amazon_df = [(compute_rnn_ip(df_new.iloc[[x]].review_body.values[0], "amazon"), df_new.iloc[[x]].star_rating.values[0]) for x in range(len(df_new))]
rnn_amazon_df = [x for x in rnn_amazon_df if np.isnan(x[0]).any() == False]

In [ ]:
with open("data/rnn_amazon.pkl", "wb") as file_obj:
    pkl.dump(rnn_amazon_df, file_obj)

del rnn_amazon_df

##### Checkpoint - 5

In [4]:
with open("data/rnn_google.pkl", "rb") as file_obj:
    rnn_google_df = pkl.load(file_obj)

In [5]:
with open("data/rnn_amazon.pkl", "rb") as file_obj:
    rnn_amazon_df = pkl.load(file_obj)

Now, we form the binary dataset and then separate both the binary and ternary datasets into inputs and labels.

In [6]:
rnn_google_bin = [x for x in rnn_google_df if x[1] == 0 or x[1] == 1]
rnn_amazon_bin = [x for x in rnn_amazon_df if x[1] == 0 or x[1] == 1]

In [7]:
rnn_google_x = np.array([x[0] for x in rnn_google_bin])
rnn_google_y = np.array([x[1] for x in rnn_google_bin])

In [8]:
rnn_amazon_x = np.array([x[0] for x in rnn_amazon_bin])
rnn_amazon_y = np.array([x[1] for x in rnn_amazon_bin])

In [9]:
rnn_google_ternary_x = np.array([x[0] for x in rnn_google_df])
rnn_google_ternary_y = np.array([x[1] for x in rnn_google_df])

In [10]:
rnn_amazon_ternary_x = np.array([x[0] for x in rnn_amazon_df])
rnn_amazon_ternary_y = np.array([x[1] for x in rnn_amazon_df])

Initial 80/20 split for train and test datasets.

In [11]:
rnn_google_x_train, rnn_google_x_test, rnn_google_y_train, rnn_google_y_test = train_test_split(rnn_google_x, rnn_google_y, test_size=0.2, random_state=42)
rnn_amazon_x_train, rnn_amazon_x_test, rnn_amazon_y_train, rnn_amazon_y_test = train_test_split(rnn_amazon_x, rnn_amazon_y, test_size=0.2, random_state=42)

In [12]:
rnn_google_x_ternary_train, rnn_google_x_ternary_test, rnn_google_y_ternary_train, rnn_google_y_ternary_test = train_test_split(rnn_google_ternary_x, rnn_google_ternary_y, test_size=0.2, random_state=42)
rnn_amazon_x_ternary_train, rnn_amazon_x_ternary_test, rnn_amazon_y_ternary_train, rnn_amazon_y_ternary_test = train_test_split(rnn_amazon_ternary_x, rnn_amazon_ternary_y, test_size=0.2, random_state=42)

80/20/20 split for train, val and test datasets.

In [13]:
rnn_google_x_train, rnn_google_x_val, rnn_google_y_train, rnn_google_y_val = train_test_split(rnn_google_x_train, rnn_google_y_train, test_size=0.2, random_state=42)
rnn_amazon_x_train, rnn_amazon_x_val, rnn_amazon_y_train, rnn_amazon_y_val = train_test_split(rnn_amazon_x_train, rnn_amazon_y_train, test_size=0.2, random_state=42)

rnn_google_x_ternary_train, rnn_google_x_ternary_val, rnn_google_y_ternary_train, rnn_google_y_ternary_val = train_test_split(rnn_google_x_ternary_train, rnn_google_y_ternary_train, test_size=0.2, random_state=42)
rnn_amazon_x_ternary_train, rnn_amazon_x_ternary_val, rnn_amazon_y_ternary_train, rnn_amazon_y_ternary_val = train_test_split(rnn_amazon_x_ternary_train, rnn_amazon_y_ternary_train, test_size=0.2, random_state=42)

Converting the numpy ndarrays to float and long tensors.

In [14]:
rnn_google_x_train, rnn_google_x_val, rnn_google_x_test, rnn_google_y_train, rnn_google_y_val, rnn_google_y_test = torch.FloatTensor(rnn_google_x_train), torch.FloatTensor(rnn_google_x_val), torch.FloatTensor(rnn_google_x_test), torch.LongTensor(rnn_google_y_train), torch.LongTensor(rnn_google_y_val), torch.LongTensor(rnn_google_y_test)
rnn_amazon_x_train, rnn_amazon_x_val, rnn_amazon_x_test, rnn_amazon_y_train, rnn_amazon_y_val, rnn_amazon_y_test = torch.FloatTensor(rnn_amazon_x_train), torch.FloatTensor(rnn_amazon_x_val), torch.FloatTensor(rnn_amazon_x_test), torch.LongTensor(rnn_amazon_y_train), torch.LongTensor(rnn_amazon_y_val), torch.LongTensor(rnn_amazon_y_test)

rnn_google_x_ternary_train, rnn_google_x_ternary_val, rnn_google_x_ternary_test, rnn_google_y_ternary_train, rnn_google_y_ternary_val, rnn_google_y_ternary_test = torch.FloatTensor(rnn_google_x_ternary_train), torch.FloatTensor(rnn_google_x_ternary_val), torch.FloatTensor(rnn_google_x_ternary_test), torch.LongTensor(rnn_google_y_ternary_train), torch.LongTensor(rnn_google_y_ternary_val), torch.LongTensor(rnn_google_y_ternary_test)
rnn_amazon_x_ternary_train, rnn_amazon_x_ternary_val, rnn_amazon_x_ternary_test, rnn_amazon_y_ternary_train, rnn_amazon_y_ternary_val, rnn_amazon_y_ternary_test = torch.FloatTensor(rnn_amazon_x_ternary_train), torch.FloatTensor(rnn_amazon_x_ternary_val), torch.FloatTensor(rnn_amazon_x_ternary_test), torch.LongTensor(rnn_amazon_y_ternary_train), torch.LongTensor(rnn_amazon_y_ternary_val), torch.LongTensor(rnn_amazon_y_ternary_test)

Deleting some variables to free-up memory.

In [15]:
del rnn_google_bin, rnn_google_df, rnn_google_x, rnn_google_y, rnn_google_ternary_x, rnn_google_ternary_y
del rnn_amazon_bin, rnn_amazon_df, rnn_amazon_x, rnn_amazon_y, rnn_amazon_ternary_x, rnn_amazon_ternary_y

Initializing the dataloaders for all datasets.

In [16]:
batch_size = 20

rnn_google_train_dataset = TensorDataset(rnn_google_x_train, rnn_google_y_train)
rnn_google_train_dataset = data(rnn_google_train_dataset)
rnn_google_val_dataset = TensorDataset(rnn_google_x_val, rnn_google_y_val)
rnn_google_val_dataset = data(rnn_google_val_dataset)
rnn_google_test_dataset = TensorDataset(rnn_google_x_test, rnn_google_y_test)
rnn_google_test_dataset = data(rnn_google_test_dataset)

rnn_google_train_loader = DataLoader(rnn_google_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
rnn_google_val_loader = DataLoader(rnn_google_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
rnn_google_test_loader = DataLoader(rnn_google_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

rnn_amazon_train_dataset = TensorDataset(rnn_amazon_x_train, rnn_amazon_y_train)
rnn_amazon_train_dataset = data(rnn_amazon_train_dataset)
rnn_amazon_val_dataset = TensorDataset(rnn_amazon_x_val, rnn_amazon_y_val)
rnn_amazon_val_dataset = data(rnn_amazon_val_dataset)
rnn_amazon_test_dataset = TensorDataset(rnn_amazon_x_test, rnn_amazon_y_test)
rnn_amazon_test_dataset = data(rnn_amazon_test_dataset)

rnn_amazon_train_loader = DataLoader(rnn_amazon_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
rnn_amazon_val_loader = DataLoader(rnn_amazon_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
rnn_amazon_test_loader = DataLoader(rnn_amazon_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

rnn_google_ternary_train_dataset = TensorDataset(rnn_google_x_ternary_train, rnn_google_y_ternary_train)
rnn_google_ternary_train_dataset = data(rnn_google_ternary_train_dataset)
rnn_google_ternary_val_dataset = TensorDataset(rnn_google_x_ternary_val, rnn_google_y_ternary_val)
rnn_google_ternary_val_dataset = data(rnn_google_ternary_val_dataset)
rnn_google_ternary_test_dataset = TensorDataset(rnn_google_x_ternary_test, rnn_google_y_ternary_test)
rnn_google_ternary_test_dataset = data(rnn_google_ternary_test_dataset)

rnn_google_ternary_train_loader = DataLoader(rnn_google_ternary_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
rnn_google_ternary_val_loader = DataLoader(rnn_google_ternary_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
rnn_google_ternary_test_loader = DataLoader(rnn_google_ternary_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

rnn_amazon_ternary_train_dataset = TensorDataset(rnn_amazon_x_ternary_train, rnn_amazon_y_ternary_train)
rnn_amazon_ternary_train_dataset = data(rnn_amazon_ternary_train_dataset)
rnn_amazon_ternary_val_dataset = TensorDataset(rnn_amazon_x_ternary_val, rnn_amazon_y_ternary_val)
rnn_amazon_ternary_val_dataset = data(rnn_amazon_ternary_val_dataset)
rnn_amazon_ternary_test_dataset = TensorDataset(rnn_amazon_x_ternary_test, rnn_amazon_y_ternary_test)
rnn_amazon_ternary_test_dataset = data(rnn_amazon_ternary_test_dataset)

rnn_amazon_ternary_train_loader = DataLoader(rnn_amazon_ternary_train_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
rnn_amazon_ternary_val_loader = DataLoader(rnn_amazon_ternary_val_dataset, batch_size = batch_size, drop_last = True, shuffle = True)
rnn_amazon_ternary_test_loader = DataLoader(rnn_amazon_ternary_test_dataset, batch_size = batch_size, drop_last = True, shuffle = True)

Declaring the class for the RNN binary classification network. Hyperparameters: learning rate - 0.0001, epochs - 20 and Adam optimizer is used across both the amazon and the google word2vec feature models for both, binary and ternary classification tasks.

In [16]:
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.hidden_size = 50
        self.n_layers = 1
        self.rnn = torch.nn.RNN(300, self.hidden_size, self.n_layers, batch_first = True)
        self.fc = torch.nn.Linear(self.hidden_size, 1)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size)
        return hidden

model = RNN()
print(model)

RNN(
  (rnn): RNN(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)


In [17]:
model = RNN()
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [18]:
epochs = 20
valid_min_loss = np.inf
train_loader = rnn_google_train_loader
val_loader = rnn_google_val_loader
test_loader = rnn_google_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/rnn_google_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.028130 	Validation Loss: 0.023682
Validation loss decreased (inf --> 0.023682). Saving model...
Epoch: 2 	Training Loss: 0.022557 	Validation Loss: 0.021149
Validation loss decreased (0.023682 --> 0.021149). Saving model...
Epoch: 3 	Training Loss: 0.021351 	Validation Loss: 0.020905
Validation loss decreased (0.021149 --> 0.020905). Saving model...
Epoch: 4 	Training Loss: 0.021047 	Validation Loss: 0.020600
Validation loss decreased (0.020905 --> 0.020600). Saving model...
Epoch: 5 	Training Loss: 0.020800 	Validation Loss: 0.020518
Validation loss decreased (0.020600 --> 0.020518). Saving model...
Epoch: 6 	Training Loss: 0.020527 	Validation Loss: 0.020908
Epoch: 7 	Training Loss: 0.020330 	Validation Loss: 0.020780
Epoch: 8 	Training Loss: 0.020230 	Validation Loss: 0.020546
Epoch: 9 	Training Loss: 0.020022 	Validation Loss: 0.019833
Validation loss decreased (0.020518 --> 0.019833). Saving model...
Epoch: 10 	Training Loss: 0.019861 	Validation Loss: 0

Clearly, the performance of the RNN model is better than all of the previous models for the google word2vec feature model for the binary classification task. This proves that, if not already apparent, that the RNN is much more complex than all the previous models. Another point to note is that the validation loss seems to be decreasing even at the 20th epoch which indicates that the model could be further improved!

In [19]:
y_pred_list = []
y_targ_list = []
test_loader = rnn_google_test_loader
model = RNN()
model.load_state_dict(torch.load('model/rnn_google_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        y_test_pred = model(inputs)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.numpy())
        y_targ_list.append(target.numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Google Word2Vec features of first 50 words for binary classification RNN model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Google Word2Vec features of first 50 words for binary classification RNN model
              precision    recall  f1-score   support

           0       0.83      0.87      0.85     20025
           1       0.86      0.82      0.84     19935

    accuracy                           0.84     39960
   macro avg       0.85      0.84      0.84     39960
weighted avg       0.85      0.84      0.84     39960



In [20]:
del model
del optimizer
del loss_fn
model = RNN()
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [21]:
epochs = 20
valid_min_loss = np.inf
train_loader = rnn_amazon_train_loader
val_loader = rnn_amazon_val_loader
test_loader = rnn_amazon_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/rnn_amazon_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.033635 	Validation Loss: 0.030491
Validation loss decreased (inf --> 0.030491). Saving model...
Epoch: 2 	Training Loss: 0.029039 	Validation Loss: 0.028165
Validation loss decreased (0.030491 --> 0.028165). Saving model...
Epoch: 3 	Training Loss: 0.026950 	Validation Loss: 0.026156
Validation loss decreased (0.028165 --> 0.026156). Saving model...
Epoch: 4 	Training Loss: 0.025527 	Validation Loss: 0.025191
Validation loss decreased (0.026156 --> 0.025191). Saving model...
Epoch: 5 	Training Loss: 0.023178 	Validation Loss: 0.021349
Validation loss decreased (0.025191 --> 0.021349). Saving model...
Epoch: 6 	Training Loss: 0.019584 	Validation Loss: 0.019112
Validation loss decreased (0.021349 --> 0.019112). Saving model...
Epoch: 7 	Training Loss: 0.018305 	Validation Loss: 0.018093
Validation loss decreased (0.019112 --> 0.018093). Saving model...
Epoch: 8 	Training Loss: 0.017701 	Validation Loss: 0.018711
Epoch: 9 	Training Loss: 0.017368 	Validation Lo

As noted with the previous models, the amazon word2vec feature model once again outperforms its counterpart.

In [22]:
y_pred_list = []
y_targ_list = []
test_loader = rnn_amazon_test_loader
model = RNN()
model.load_state_dict(torch.load('model/rnn_amazon_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        y_test_pred = model(inputs)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.numpy())
        y_targ_list.append(target.numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Amazon Word2Vec features of first 50 words for binary classification RNN model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Amazon Word2Vec features of first 50 words for binary classification RNN model
              precision    recall  f1-score   support

           0       0.86      0.85      0.85     19992
           1       0.85      0.86      0.86     19948

    accuracy                           0.86     39940
   macro avg       0.86      0.86      0.86     39940
weighted avg       0.86      0.86      0.86     39940



In [151]:
class RNN_ternary(torch.nn.Module):
    def __init__(self):
        super(RNN_ternary, self).__init__()
        self.hidden_size = 50
        self.n_layers = 1
        self.rnn = torch.nn.RNN(300, self.hidden_size, self.n_layers, batch_first = True)
        self.fc = torch.nn.Linear(self.hidden_size, 3)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size)
        return hidden

model = RNN_ternary()
print(model)

RNN_ternary(
  (rnn): RNN(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


Once again, we are initializing class weights for proper performance of the model.

In [19]:
np_ary = rnn_google_y_ternary_train.numpy()
class_weights_rnn_google = torch.tensor(class_weight.compute_class_weight('balanced', np.unique(np_ary), np_ary))
print("Class weights of ternary classification for Google word2vec feature model:")
print(class_weights_rnn_google)
np_ary = rnn_amazon_y_ternary_train.numpy()
class_weights_rnn_amazon = torch.tensor(class_weight.compute_class_weight('balanced', np.unique(np_ary), np_ary))
print("\nClass weights of ternary classification for Amazon word2vec feature model:")
print(class_weights_rnn_amazon)

Class weights of ternary classification for Google word2vec feature model:
tensor([0.8297, 0.8367, 1.6678], dtype=torch.float64)

Class weights of ternary classification for Amazon word2vec feature model:
tensor([0.8303, 0.8372, 1.6635], dtype=torch.float64)


In [187]:
del model
del optimizer
del loss_fn
model = RNN_ternary()
loss_fn = torch.nn.CrossEntropyLoss(weight = class_weights_rnn_google)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [188]:
epochs = 20
valid_min_loss = np.inf
train_loader = rnn_google_ternary_train_loader
val_loader = rnn_google_ternary_val_loader
test_loader = rnn_google_ternary_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1).double(), target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output.squeeze(1).double(), target)
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/rnn_google_ternary_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.050586 	Validation Loss: 0.046852
Validation loss decreased (inf --> 0.046852). Saving model...
Epoch: 2 	Training Loss: 0.046015 	Validation Loss: 0.045228
Validation loss decreased (0.046852 --> 0.045228). Saving model...
Epoch: 3 	Training Loss: 0.045355 	Validation Loss: 0.045180
Validation loss decreased (0.045228 --> 0.045180). Saving model...
Epoch: 4 	Training Loss: 0.045016 	Validation Loss: 0.044481
Validation loss decreased (0.045180 --> 0.044481). Saving model...
Epoch: 5 	Training Loss: 0.044509 	Validation Loss: 0.043520
Validation loss decreased (0.044481 --> 0.043520). Saving model...
Epoch: 6 	Training Loss: 0.043681 	Validation Loss: 0.043697
Epoch: 7 	Training Loss: 0.043380 	Validation Loss: 0.043219
Validation loss decreased (0.043520 --> 0.043219). Saving model...
Epoch: 8 	Training Loss: 0.043116 	Validation Loss: 0.042657
Validation loss decreased (0.043219 --> 0.042657). Saving model...
Epoch: 9 	Training Loss: 0.042881 	Validation Lo

From the below stats, it can be noted that there is a slight increase in the performance when compared to previous models for the ternary classification task. That being said, overall, the performance is still mediocre. The reasons for such performance continue to remain the same as mentioned before (improper dataset or not-so-complex model).

In [189]:
y_pred_list = []
y_targ_list = []
test_loader = rnn_google_ternary_test_loader
model = RNN_ternary()
model.load_state_dict(torch.load('model/rnn_google_ternary_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.numpy())
        y_targ_list.append(target.numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Google Word2Vec features of fist 50 words for ternary classification RNN model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Google Word2Vec features of fist 50 words for ternary classification RNN model
              precision    recall  f1-score   support

           0       0.77      0.65      0.70     19791
           1       0.81      0.68      0.74     20168
           2       0.33      0.54      0.41      9981

    accuracy                           0.64     49940
   macro avg       0.64      0.62      0.62     49940
weighted avg       0.70      0.64      0.66     49940



In [207]:
del model
del optimizer
del loss_fn
model = RNN_ternary()
loss_fn = torch.nn.CrossEntropyLoss(weight = class_weights_rnn_amazon)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [208]:
epochs = 20
valid_min_loss = np.inf
train_loader = rnn_amazon_ternary_train_loader
val_loader = rnn_amazon_ternary_val_loader
test_loader = rnn_amazon_ternary_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1).double(), target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output.squeeze(1).double(), target)
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/rnn_amazon_ternary_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.052677 	Validation Loss: 0.050518
Validation loss decreased (inf --> 0.050518). Saving model...
Epoch: 2 	Training Loss: 0.049854 	Validation Loss: 0.048954
Validation loss decreased (0.050518 --> 0.048954). Saving model...
Epoch: 3 	Training Loss: 0.048552 	Validation Loss: 0.048463
Validation loss decreased (0.048954 --> 0.048463). Saving model...
Epoch: 4 	Training Loss: 0.047926 	Validation Loss: 0.047600
Validation loss decreased (0.048463 --> 0.047600). Saving model...
Epoch: 5 	Training Loss: 0.046806 	Validation Loss: 0.046919
Validation loss decreased (0.047600 --> 0.046919). Saving model...
Epoch: 6 	Training Loss: 0.045091 	Validation Loss: 0.044774
Validation loss decreased (0.046919 --> 0.044774). Saving model...
Epoch: 7 	Training Loss: 0.044653 	Validation Loss: 0.045930
Epoch: 8 	Training Loss: 0.045074 	Validation Loss: 0.045071
Epoch: 9 	Training Loss: 0.044252 	Validation Loss: 0.045015
Epoch: 10 	Training Loss: 0.044645 	Validation Loss: 0

In [209]:
y_pred_list = []
y_targ_list = []
test_loader = rnn_amazon_ternary_test_loader
model = RNN_ternary()
model.load_state_dict(torch.load('model/rnn_amazon_ternary_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.numpy())
        y_targ_list.append(target.numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Amazon Word2Vec features of fist 50 words for ternary classification RNN model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Amazon Word2Vec features of fist 50 words for ternary classification RNN model
              precision    recall  f1-score   support

           0       0.74      0.64      0.69     19865
           1       0.74      0.68      0.71     20087
           2       0.30      0.43      0.36      9968

    accuracy                           0.62     49920
   macro avg       0.59      0.59      0.58     49920
weighted avg       0.65      0.62      0.63     49920



## GRU Model

Declaring the class for the GRU binary classification network. The hyperparameters for the GRU models (both amazon and google word2vec feature models) for both tasks (binary and ternary classification) remain the same as the RNN models. Learning rate - 0.0001, epochs - 20 and Adam optimizer. We even use the same dataloaders as the ones that were used with the RNN models since the data for both networks are the same.

In [210]:
class GRU(torch.nn.Module):
    def __init__(self):
        super(GRU, self).__init__()
        self.hidden_size = 50
        self.n_layers = 1
        self.rnn = torch.nn.GRU(300, self.hidden_size, self.n_layers, batch_first = True)
        self.fc = torch.nn.Linear(self.hidden_size, 1)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size)
        return hidden

model = GRU()
print(model)

GRU(
  (rnn): GRU(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [211]:
del model
del optimizer
del loss_fn
model = GRU()
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [212]:
epochs = 20
valid_min_loss = np.inf
train_loader = rnn_google_train_loader
val_loader = rnn_google_val_loader
test_loader = rnn_google_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/gru_google_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.024937 	Validation Loss: 0.019186
Validation loss decreased (inf --> 0.019186). Saving model...
Epoch: 2 	Training Loss: 0.018449 	Validation Loss: 0.017737
Validation loss decreased (0.019186 --> 0.017737). Saving model...
Epoch: 3 	Training Loss: 0.017251 	Validation Loss: 0.016776
Validation loss decreased (0.017737 --> 0.016776). Saving model...
Epoch: 4 	Training Loss: 0.016481 	Validation Loss: 0.016210
Validation loss decreased (0.016776 --> 0.016210). Saving model...
Epoch: 5 	Training Loss: 0.015913 	Validation Loss: 0.015737
Validation loss decreased (0.016210 --> 0.015737). Saving model...
Epoch: 6 	Training Loss: 0.015534 	Validation Loss: 0.015436
Validation loss decreased (0.015737 --> 0.015436). Saving model...
Epoch: 7 	Training Loss: 0.015141 	Validation Loss: 0.015117
Validation loss decreased (0.015436 --> 0.015117). Saving model...
Epoch: 8 	Training Loss: 0.014885 	Validation Loss: 0.014931
Validation loss decreased (0.015117 --> 0.014931

From the results below, it is apparent that there is a considerable improvement in the performance. This indicates that the GRU is the most robust model of all the models used here.

In [213]:
y_pred_list = []
y_targ_list = []
test_loader = rnn_google_test_loader
model = GRU()
model.load_state_dict(torch.load('model/gru_google_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        y_test_pred = model(inputs)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.numpy())
        y_targ_list.append(target.numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Google Word2Vec features of first 50 words for binary classification GRU model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Google Word2Vec features of first 50 words for binary classification GRU model
              precision    recall  f1-score   support

           0       0.88      0.88      0.88     20025
           1       0.88      0.88      0.88     19935

    accuracy                           0.88     39960
   macro avg       0.88      0.88      0.88     39960
weighted avg       0.88      0.88      0.88     39960



In [214]:
del model
del optimizer
del loss_fn
model = GRU()
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [215]:
epochs = 20
valid_min_loss = np.inf
train_loader = rnn_amazon_train_loader
val_loader = rnn_amazon_val_loader
test_loader = rnn_amazon_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs, target
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output.squeeze(1), target.float())
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/gru_amazon_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.026783 	Validation Loss: 0.019479
Validation loss decreased (inf --> 0.019479). Saving model...
Epoch: 2 	Training Loss: 0.017644 	Validation Loss: 0.016922
Validation loss decreased (0.019479 --> 0.016922). Saving model...
Epoch: 3 	Training Loss: 0.015976 	Validation Loss: 0.016283
Validation loss decreased (0.016922 --> 0.016283). Saving model...
Epoch: 4 	Training Loss: 0.015053 	Validation Loss: 0.015845
Validation loss decreased (0.016283 --> 0.015845). Saving model...
Epoch: 5 	Training Loss: 0.014405 	Validation Loss: 0.015565
Validation loss decreased (0.015845 --> 0.015565). Saving model...
Epoch: 6 	Training Loss: 0.013897 	Validation Loss: 0.015203
Validation loss decreased (0.015565 --> 0.015203). Saving model...
Epoch: 7 	Training Loss: 0.013469 	Validation Loss: 0.015192
Validation loss decreased (0.015203 --> 0.015192). Saving model...
Epoch: 8 	Training Loss: 0.013104 	Validation Loss: 0.015200
Epoch: 9 	Training Loss: 0.012769 	Validation Lo

The performance of the amazon word2vec feature model is almost the same as the performance of its counterpart, the google model.

In [216]:
y_pred_list = []
y_targ_list = []
test_loader = rnn_amazon_test_loader
model = GRU()
model.load_state_dict(torch.load('model/gru_amazon_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        y_test_pred = model(inputs)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.numpy())
        y_targ_list.append(target.numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Amazon Word2Vec features of first 50 words for binary classification GRU model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Amazon Word2Vec features of first 50 words for binary classification GRU model
              precision    recall  f1-score   support

           0       0.89      0.87      0.88     19992
           1       0.87      0.89      0.88     19948

    accuracy                           0.88     39940
   macro avg       0.88      0.88      0.88     39940
weighted avg       0.88      0.88      0.88     39940



In [17]:
class GRU_ternary(torch.nn.Module):
    def __init__(self):
        super(GRU_ternary, self).__init__()
        self.hidden_size = 50
        self.n_layers = 1
        self.rnn = torch.nn.GRU(300, self.hidden_size, self.n_layers, batch_first = True)
        self.fc = torch.nn.Linear(self.hidden_size, 3)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size).to(device)
        return hidden

model = GRU_ternary()
print(model)

GRU_ternary(
  (rnn): GRU(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=3, bias=True)
)


In [20]:
del model
del optimizer
del loss_fn
model = GRU_ternary().to(device)
loss_fn = torch.nn.CrossEntropyLoss(weight = class_weights_rnn_google).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [21]:
epochs = 20
valid_min_loss = np.inf
train_loader = rnn_google_ternary_train_loader
val_loader = rnn_google_ternary_val_loader
test_loader = rnn_google_ternary_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1).double(), target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs.to(device), target.to(device)
        output = model(inputs)
        loss = loss_fn(output.squeeze(1).double(), target)
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/gru_google_ternary_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.047073 	Validation Loss: 0.042103
Validation loss decreased (inf --> 0.042103). Saving model...
Epoch: 2 	Training Loss: 0.041609 	Validation Loss: 0.040450
Validation loss decreased (0.042103 --> 0.040450). Saving model...
Epoch: 3 	Training Loss: 0.040378 	Validation Loss: 0.039572
Validation loss decreased (0.040450 --> 0.039572). Saving model...
Epoch: 4 	Training Loss: 0.039569 	Validation Loss: 0.038902
Validation loss decreased (0.039572 --> 0.038902). Saving model...
Epoch: 5 	Training Loss: 0.038903 	Validation Loss: 0.038592
Validation loss decreased (0.038902 --> 0.038592). Saving model...
Epoch: 6 	Training Loss: 0.038358 	Validation Loss: 0.038103
Validation loss decreased (0.038592 --> 0.038103). Saving model...
Epoch: 7 	Training Loss: 0.037742 	Validation Loss: 0.037845
Validation loss decreased (0.038103 --> 0.037845). Saving model...
Epoch: 8 	Training Loss: 0.037216 	Validation Loss: 0.037869
Epoch: 9 	Training Loss: 0.036839 	Validation Lo

As seen in the binary classification task, GRU proves to be the most robust model even for ternary classification as indicated by the stats below. The F1 scores of all the classes have risen considerbaly, meaning that GRU is better at identifying the underlying patterns in the ternary classification task than all other previous models. The validation loss seems to be decreasing further even after 20 epochs showing that there is room for improvement with this model. Having said that, the performance, although improved, is still not great. This proves that the there is a bit of ambiguity in the dataset when it comes to the neutral class such that even the most complex model is not able to acheive good performance. Further, since the validation loss seem to be decreasing at the 20th epoch we can probably guess with some level of confidence that more training would lead to better results, proving that a more complex model could in fact properly model the relation between the word vectors and the three classes.

In [24]:
y_pred_list = []
y_targ_list = []
test_loader = rnn_google_ternary_test_loader
model = GRU_ternary().to(device)
model.load_state_dict(torch.load('model/gru_google_ternary_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Google Word2Vec features of fist 50 words for ternary classification GRU model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Google Word2Vec features of fist 50 words for ternary classification GRU model
              precision    recall  f1-score   support

           0       0.78      0.73      0.75     19794
           1       0.83      0.74      0.79     20168
           2       0.39      0.53      0.45      9978

    accuracy                           0.70     49940
   macro avg       0.67      0.67      0.66     49940
weighted avg       0.73      0.70      0.71     49940



In [25]:
del rnn_google_train_dataset, rnn_google_val_dataset, rnn_google_test_dataset, rnn_google_ternary_train_dataset, rnn_google_ternary_val_dataset, rnn_google_ternary_test_dataset
del rnn_amazon_train_dataset, rnn_amazon_val_dataset, rnn_amazon_test_dataset

In [26]:
del model
del optimizer
del loss_fn
model = GRU_ternary().to(device)
loss_fn = torch.nn.CrossEntropyLoss(weight = class_weights_rnn_google).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [27]:
epochs = 20
valid_min_loss = np.inf
train_loader = rnn_amazon_ternary_train_loader
val_loader = rnn_amazon_ternary_val_loader
test_loader = rnn_amazon_ternary_test_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output.squeeze(1).double(), target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in val_loader:
        inputs, target = inputs.to(device), target.to(device)
        output = model(inputs)
        loss = loss_fn(output.squeeze(1).double(), target)
        valid_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(val_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    if valid_loss <= valid_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_min_loss, valid_loss))
        torch.save(model.state_dict(), 'model/gru_amazon_ternary_model.pt')
        valid_min_loss = valid_loss

Epoch: 1 	Training Loss: 0.050945 	Validation Loss: 0.043778
Validation loss decreased (inf --> 0.043778). Saving model...
Epoch: 2 	Training Loss: 0.040967 	Validation Loss: 0.039543
Validation loss decreased (0.043778 --> 0.039543). Saving model...
Epoch: 3 	Training Loss: 0.038641 	Validation Loss: 0.038567
Validation loss decreased (0.039543 --> 0.038567). Saving model...
Epoch: 4 	Training Loss: 0.037624 	Validation Loss: 0.038254
Validation loss decreased (0.038567 --> 0.038254). Saving model...
Epoch: 5 	Training Loss: 0.036928 	Validation Loss: 0.037961
Validation loss decreased (0.038254 --> 0.037961). Saving model...
Epoch: 6 	Training Loss: 0.036330 	Validation Loss: 0.037564
Validation loss decreased (0.037961 --> 0.037564). Saving model...
Epoch: 7 	Training Loss: 0.035856 	Validation Loss: 0.037778
Epoch: 8 	Training Loss: 0.035378 	Validation Loss: 0.037494
Validation loss decreased (0.037564 --> 0.037494). Saving model...
Epoch: 9 	Training Loss: 0.034953 	Validation Lo

Lastly, again the performance of the google and the amazon word2vec feature models are comparable, and one could even say google model's performance is better in this situation.

In [28]:
y_pred_list = []
y_targ_list = []
test_loader = rnn_amazon_ternary_test_loader
model = GRU_ternary().to(device)
model.load_state_dict(torch.load('model/gru_amazon_ternary_model.pt'))
model.eval()
with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]
print("Statistics of Amazon Word2Vec features of fist 50 words for ternary classification GRU model")
print(classification_report(y_targ_list, y_pred_list))

Statistics of Amazon Word2Vec features of fist 50 words for ternary classification GRU model
              precision    recall  f1-score   support

           0       0.81      0.68      0.74     19871
           1       0.82      0.74      0.78     20085
           2       0.37      0.56      0.45      9964

    accuracy                           0.68     49920
   macro avg       0.67      0.66      0.66     49920
weighted avg       0.73      0.68      0.70     49920



Final conclusion: GRU the most complex and robust of all the models tested. Dataset for ternary classification isn't proper due to the ambiguous nature of the neutral class reviews making it hard for all models to produce good results. 